In [1]:
import numpy as np

In [2]:
npzdata=np.load("data.npz")

In [3]:
npzdata

In [4]:
data=npzdata['data']
labels=npzdata['labels']

In [5]:
data

array([[ 1,  1, -1, ..., -1, -1, -1],
       [ 1,  1, -1, ..., -1, -1, -1],
       [ 1,  1, -1, ..., -1, -1, -1],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int16)

In [6]:
labels

array([u'9gag.com', u'9gag.com', u'9gag.com', ..., None, None, None],
      dtype=object)

In [7]:
data.shape

(249996, 5000)